1. 키워드 입력
2. naver_api에서 뉴스 수집
3. gemini로 뉴스 수집결과 요약해서 출력하기

In [16]:
# ------------------------------
# 네이버 뉴스 검색 + Gemini 요약
# ------------------------------

import urllib.request
import urllib.parse
import json
import google.generativeai as genai


# ------------------------------
# 🔍 0. 검색 키워드 입력
# ------------------------------
query = input("검색할 키워드를 입력하세요: ").strip()
if not query:
    print("키워드를 입력하지 않았습니다. 프로그램을 종료합니다.")
    exit()

print(f"\n▶ 검색 키워드: {query}\n")


# ------------------------------
# 1. 네이버 뉴스 API
# ------------------------------

NAVER_CLIENT_ID = ""         # -> 네이버api id
NAVER_CLIENT_SECRET = ""     # -> 네이버api pw

encText = urllib.parse.quote(query)

url = f"https://openapi.naver.com/v1/search/news.json?query={encText}&display=10"

request = urllib.request.Request(url)
request.add_header("X-Naver-Client-Id", NAVER_CLIENT_ID)
request.add_header("X-Naver-Client-Secret", NAVER_CLIENT_SECRET)

response = urllib.request.urlopen(request)
rescode = response.getcode()

if rescode == 200:
    result = json.loads(response.read().decode('utf-8'))
else:
    print("Naver API Error:", rescode)
    exit()


# ------------------------------
# 2. 뉴스 제목 + 링크 추출
# ------------------------------

news_items = result.get("items", [])

news_list_text = ""
for item in news_items:
    title = item["title"].replace("<b>", "").replace("</b>", "")
    link = item["link"]
    news_list_text += f"- 제목: {title}\n  링크: {link}\n\n"

print("📌 수집된 뉴스 목록:")
print(news_list_text)


# ------------------------------
# 3. Gemini API 설정
# ------------------------------

GEMINI_API_KEY = "" #  -> gemivi key입력
genai.configure(api_key=GEMINI_API_KEY)

model = genai.GenerativeModel("gemini-2.5-flash")


# ------------------------------
# 4. Gemini 요약 요청
# ------------------------------

prompt = f"""
다음은 '{query}' 키워드로 검색한 네이버 뉴스 결과입니다.
각 뉴스의 핵심 내용을 요약하고 전체 흐름을 정리해줘.

뉴스 목록:
{news_list_text}
"""

response = model.generate_content(prompt)

print("\n\n📝 Gemini 요약 결과:")
print(response.text)


검색할 키워드를 입력하세요: 핀테크

▶ 검색 키워드: 핀테크

📌 수집된 뉴스 목록:
- 제목: 한국전자인증 &quot;개인정보 보호 위해 '패스키' 확산 추진&quot;
  링크: https://www.datanet.co.kr/news/articleView.html?idxno=207509

- 제목: [2026 금융 전망]② 스테이블코인 부상하면 은행 유동성 긴장…딜로이트...
  링크: https://www.newsfreezone.co.kr/news/articleView.html?idxno=662347

- 제목: 세종대 김대종 교수, 노원구상공회 CEO과정 특강… &quot;AI·클라우드 융합...
  링크: https://www.pinpointnews.co.kr/news/articleView.html?idxno=403820

- 제목: &quot;금융 카르텔 무섭네&quot; 원화 스테이블코인 공표 임박...은행 중심 체계에...
  링크: https://www.techm.kr/news/articleView.html?idxno=147526

- 제목: 코빗 &quot;비트코인 유동성 사이클 재편...2026년 추가 고점 가능성&quot;
  링크: http://theviewers.co.kr/View.aspx?No=3884452

- 제목: JP모건 '정치 성향 계좌 해지' 논란…암호화폐 업계 '초크포인트 2.0' 우...
  링크: https://www.tokenpost.kr/news/cryptocurrency/312271

- 제목: 씨지인사이드, 창업진흥원 ‘이노웨이브 리그’ 최우수상 수상
  링크: https://www.venturesquare.net/1018366

- 제목: 로빈후드, 인도네시아 암호화폐 시장 진출…현지 인가 플랫폼 2곳 인수
  링크: https://www.tokenpost.kr/news/cryptocurrency/312264

- 제목: 코빗 리서치센터, '2026 시장 전망' 보고서 공개…스테이블코인 인프라...
  링크

In [6]:
#!pip install google-generativeai

In [17]:
import urllib.request
import urllib.parse
import json
import google.generativeai as genai
import gradio as gr


# ------------------------------
# ✔ Gemini API 설정
# ------------------------------
GEMINI_API_KEY = ""    #  -> gemivi key입력
genai.configure(api_key=GEMINI_API_KEY)
model = genai.GenerativeModel("gemini-2.5-flash")


# ------------------------------
# ✔ 네이버 뉴스 API 설정
# ------------------------------
NAVER_CLIENT_ID = ""       # -> 네이버api id
NAVER_CLIENT_SECRET = ""   # -> 네이버api pw


# ------------------------------
# ✔ 뉴스 검색 + 요약 함수
# ------------------------------
def search_and_summarize(keyword):

    if not keyword.strip():
        return "❗ 키워드를 입력하세요.", ""

    # Encode
    encText = urllib.parse.quote(keyword)
    url = f"https://openapi.naver.com/v1/search/news.json?query={encText}&display=10"

    # Request
    request = urllib.request.Request(url)
    request.add_header("X-Naver-Client-Id", NAVER_CLIENT_ID)
    request.add_header("X-Naver-Client-Secret", NAVER_CLIENT_SECRET)

    response = urllib.request.urlopen(request)
    rescode = response.getcode()

    if rescode != 200:
        return f"네이버 API 오류 발생: {rescode}", ""

    result = json.loads(response.read().decode('utf-8'))
    news_items = result.get("items", [])

    # 뉴스 제목 + 링크 정리
    news_list_text = ""
    for item in news_items:
        title = item["title"].replace("<b>", "").replace("</b>", "")
        link = item["link"]
        news_list_text += f"- 제목: {title}\n  링크: {link}\n\n"

    if not news_list_text:
        return "검색 결과가 없습니다.", ""

    # Gemini 요약
    prompt = f"""
    다음은 '{keyword}' 키워드로 검색한 네이버 뉴스 목록입니다.
    주요 내용을 정리하고 전체 흐름을 요약해줘.

    {news_list_text}
    """

    response = model.generate_content(prompt)
    summary = response.text

    return news_list_text, summary


# ------------------------------
# ✔ Gradio UI 구성
# ------------------------------
with gr.Blocks(title="네이버 뉴스 요약기") as demo:

    gr.Markdown("## 🔍 네이버 뉴스 자동 요약기 (Gemini 2.5-flash)")
    gr.Markdown("키워드를 입력하면 뉴스 10개를 수집해 Gemini가 자동으로 요약합니다.")

    keyword_input = gr.Textbox(label="검색 키워드 입력", placeholder="예: 핀테크, AI, 경제, 부동산 ...")

    search_button = gr.Button("검색 & 요약 실행")

    news_output = gr.Textbox(label="📌 수집된 뉴스 목록", lines=15)
    summary_output = gr.Textbox(label="📝 Gemini 요약 결과", lines=15)

    search_button.click(
        search_and_summarize,
        inputs=keyword_input,
        outputs=[news_output, summary_output]
    )


# 실행
demo.launch()


* Running on local URL:  http://127.0.0.1:7860
* To create a public link, set `share=True` in `launch()`.
